In [20]:
# Import the sklearn models from MLflow
import mlflow.sklearn
import mlflow
from pprint import pprint
from sklearn.ensemble import RandomForestRegressor

In [19]:
run_name = "mlflow-models-and-registry"

# -- MLflow Models

# log model to current run
with mlflow.start_run(run_name=run_name) as run:
    params = {"n_estimators": 4, "random_state": 42}
    rfr = RandomForestRegressor(**params).fit([[0, 1, 0]], [1])
    mlflow.log_params(params)
    # TODO: Log the model rfr and give it the path "sklearn-model"
    mlflow.sklearn.log_model(rfr, artifact_path="sklearn-model")

    # TODO: modify the model_uri to load from the path "sklearn-model"
    model_uri = f"runs:/{run.info.run_id}/sklearn-model"

# TODO: Load the model using the model uri to test it
model = mlflow.sklearn.load_model(model_uri=model_uri)

data = [[0, 1, 0]]
model_pred = model.predict(data)
print(f"model_pred: {model_pred}")


model_pred: [1.]


/Users/sebastian.blum/.pyenv/versions/3.10.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/Users/sebastian.blum/.pyenv/versions/3.10.10/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


In [21]:


# -- MLflow Model Registry

model_name = "sklearn-model-name"
# register model in mlflow registry
# TODO: Register the model to the model registry using the model_uri from before and the model_name
mv = mlflow.register_model(model_uri, model_name)
print("Name: {}".format(mv.name))
print("Version: {}".format(mv.version))
print("Stage: {}".format(mv.current_stage))


# TODO: modify the model_uri to load the model from the registry
model_uri = f"models:/{model_name}/{mv.version}"
model = mlflow.sklearn.load_model(model_uri=model_uri)

data = [[0, 1, 0]]
model_pred = model.predict(data)
print(f"model_pred: {model_pred}")


# Transition the model to another stage
from mlflow import MlflowClient

client = MlflowClient()

# TODO: Transition the model to the stage "Production"
stage = "Staging"  # None, Production

client.transition_model_version_stage(name=model_name, version=mv.version, stage=stage)

# print registered models
for rm in client.search_registered_models():
    pprint(dict(rm), indent=4)


Registered model 'sklearn-model-name' already exists. Creating a new version of this model...
2023/07/18 12:07:21 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: sklearn-model-name, version 6


Name: sklearn-model-name
Version: 6
Stage: None
model_pred: [1.]
{   'aliases': {},
    'creation_timestamp': 1689674536078,
    'description': None,
    'last_updated_timestamp': 1689674841762,
    'latest_versions': [   <ModelVersion: aliases=[], creation_timestamp=1689674841728, current_stage='Staging', description=None, last_updated_timestamp=1689674841762, name='sklearn-model-name', run_id='e7e80289c8d643bdad79a24d50fd4550', run_link=None, source='file:///Users/sebastian.blum/Documents/Practice/mlflow-training/03-MLflow-Models-and-Registry/exercise/mlruns/0/e7e80289c8d643bdad79a24d50fd4550/artifacts/sklearn-model', status='READY', status_message=None, tags={}, user_id=None, version=6>,
                           <ModelVersion: aliases=[], creation_timestamp=1689674585640, current_stage='None', description=None, last_updated_timestamp=1689674585640, name='sklearn-model-name', run_id=None, run_link=None, source='models:/sklearn-model-name/1', status='READY', status_message=None, tag

Created version '6' of model 'sklearn-model-name'.
